# Process MPC style tables for Uniview data module
Read in and process the data from these MPC sites http://www.minorplanetcenter.net/iau/lists/t_tnos.html and http://www.minorplanetcenter.net/iau/lists/t_centaurs.html

Through some black magic Melissa Brucker has scraped those web pages and output the data as excell spreadsheets. I've exported those as .csv files

In [142]:
#Imports
from astropy.table import Table,Column
from astropy.time import Time
from astropy import units
from astropy.io import ascii

In [161]:
#Read in .csv file
MPC_table = ascii.read('MPC_CentaurSDOlist_20160126.csv')
#Convert the stupid (and inconsistent!) date formats that MPC uses, write them out as mjd
epochTimeList=[]
epochList=MPC_table['Epoch']
for epochInt in epochList:
    epoch=str(epochInt)
    epochString= '{}-{}-{}'.format(epoch[0:4],epoch[4:6],epoch[6:8])
    epochTime = Time(epochString).mjd
    epochTimeList.append(epochTime)
timeList=[]
discoveryList=MPC_table['Discovery date']
for discovery in discoveryList:
    elements = discovery.split('/')
    discoveryString= '{}-{:02d}-{:02d}'.format(elements[2],int(elements[0]),int(elements[1]))
    discoveryTime = Time(discoveryString).mjd
    timeList.append(discoveryTime)
MPC_table.remove_columns(['Discovery date','Epoch'])
MPC_table.add_column(Column(timeList,name='Discovery'))
MPC_table.add_column(Column(epochTimeList,name='Epoch'))

Add a column for daily motion. Calculate the Mean anomoly at the J2000 epoch (what the graphics shaders are expecting). the J2000 mjd is 51544.5

In [162]:
#Calculate daily motion via Kepler's Third Law
MPC_table.add_column(Column((360/365.25)*MPC_table['a']**(-3./2.),name='n'))
#Shift the Mean Anomoly to the J2000 epoch
MPC_table['M']=MPC_table['M'] + (51544.5 - MPC_table['Epoch']) * MPC_table['n']

## Filter the table by Opservation Arc

The quantity we have available in the MPC table is:

Opps. = Number of oppositions at which the object has been observed. If observations have been made at one opposition only, the arc length in days is given in parentheses.  

We will filter out everything with less than one opposition of observations

In [163]:
deleteList=[]
for i,row in enumerate(MPC_table):
  if ('(' in row['Opps.'] and not('2013 RF98' in row['Prov.Des.'])):
    deleteList.append(i)
MPC_table.remove_rows(deleteList)


In [147]:
MPC_table.show_in_browser(jsviewer=True)

## Add a special column used to highlight specific objects
This is a hack for the Mike Brown lecture data set. At some point I should build a module without it

In [136]:
classList = []
for row in MPC_table:
  c = 1
  if ('Eris' in row['Designation (and name)']):
    c=2
  elif ('Haumea' in row['Designation (and name)']):
    c=3
  elif ('Sedna' in row['Designation (and name)']):
    c=4
  classList.append(c)

MPC_table.add_column(Column(classList, name='class'))

In [82]:
MPC_table.show_in_browser(jsviewer=True)

## Match elongated orbits a>250 AU, q >30 AU

In [186]:
count =0
FarTable=[]
for row in MPC_table:
    if (row['a']>250.0 and row['q']>30.):
        if (count==0):
            FarTable=Table(row)
        else:
            FarTable.add_row(row)
        count=count+1
        
#Precession rates from Dan Fabrycky
Precess=dict([('2013 RF98',0.86),('2012 VP113',0.27),('2010 GB174',0.06),('2007 TG422',0.0),('2004 VN112',-0.25),('2003 VB12',0.13)])

PrecessList=[]
for row in FarTable:
    PrecessList.append(Precess[row['Prov.Des.']])
FarTable.add_column(Column(PrecessList,name="Precess"))

FarTable

Designation (and name),Prov.Des.,q,Q,H,M,Peri.,Node,Incl.,e,a,Opps.,Ref.,site,discoverer(s),Discovery,Epoch,n,Precess
string144,string80,float64,float64,float64,float64,float64,float64,float64,float64,float64,string48,string80,string32,string464,float64,float64,float64,float64
--,2013 RF98,36.29,614.0,8.6,-0.743726071964,316.5,67.6,29.6,0.888,325.0,( 56d),MPO 331342,W84,Dark Energy Survey,56547.0,56560.0,0.000168223720858,0.86
0.0,2012 VP113,80.29,444.0,4.0,1.93910592116,292.9,90.8,24.0,0.694,262.0,3,MPO 289737,807,0.0,56236.0,57400.0,0.000232412958559,0.27
0.0,2010 GB174,48.719,694.0,6.5,2.39236450606,347.8,130.6,21.5,0.869,371.0,4,MPO 262140,568,0.0,55298.0,57400.0,0.000137927673801,0.06
0.0,2007 TG422,35.579,949.0,6.2,-0.228845249241,285.8,113.0,18.6,0.928,492.0,4,MPO 252047,705,"A. C. Becker, A. W. Puckett, J. Kubica",54376.0,57400.0,9.03159848417e-05,0.0
0.0,2004 VN112,47.327,594.0,6.4,-0.608210680871,327.2,66.0,25.6,0.852,320.0,6,MPO 357120,807,0.0,53315.0,57400.0,0.000172181825783,-0.25
(90377) Sedna,2003 VB12,76.045,939.0,1.6,357.594449758,311.5,144.5,11.9,0.85,507.0,13,MPO 359374,675,"M. E. Brown, C. A. Trujillo, D. L. Rabinowitz",52957.0,57400.0,8.63376725873e-05,0.13


## Perpendicular Centaurs a>250 AU, i > 50

In [194]:
import numpy as np
count =0
PCentaurTable=[]
for row in MPC_table:
    if (row['a']>200.0 and row['Incl.']>50.):
        if (count==0):
            PCentaurTable=Table(row)
        else:
            PCentaurTable.add_row(row)
        count=count+1
# Add a column of zeros for precession rates, which we do not have yet.
PCentaurTable.add_column(Column(np.zeros(len(PCentaurTable)),name="Precess"))

PCentaurTable

Designation (and name),Prov.Des.,q,Q,H,M,Peri.,Node,Incl.,e,a,Opps.,Ref.,site,discoverer(s),Discovery,Epoch,n,Precess
string144,string80,float64,float64,float64,float64,float64,float64,float64,float64,float64,string48,string80,string32,string464,float64,float64,float64,float64
--,2013 BL76,8.371,2418.0,10.8,-0.136610808046,166.1,180.2,98.6,0.993,1213.0,3,MPO 307057,G96,Mt. Lemmon Survey,56312.0,57400.0,2.33303403716e-05,0.0
0.0,2012 DR30,14.551,2794.0,7.1,-0.109704792363,195.4,341.4,78.0,0.99,1404.0,8,MPO 339460,E12,Siding Spring Survey,55979.0,57400.0,1.87353415358e-05,0.0
0.0,2010 BK118,6.1,963.0,10.2,-0.442011147845,179.1,176.0,143.9,0.987,484.0,5,MPO 271268,C51,WISE,55226.0,57400.0,9.25644518565e-05,0.0
-418993,2009 MS9,11.002,684.0,9.9,-0.689011534057,128.7,220.2,68.1,0.968,348.0,5,MPO 360982,568,"J.-M. Petit, B. Gladman, J. J. Kavelaars",55007.0,57400.0,0.000151825042107,0.0
-336756,2010 NV1,9.41,637.0,10.6,-0.694197067899,132.9,136.2,140.8,0.971,323.0,7,MPO 345897,C51,WISE,55378.0,57400.0,0.00016978858644,0.0


## Output Uniview .raw style file
nine floats per line, our TNO shader expects:

a  e  i  Omega  omega  M  H  class  Discobery(mjd)

In [137]:
#Main Table
outTable = Table([MPC_table['a'],MPC_table['e'],MPC_table['Incl.'],MPC_table['Node'],MPC_table['Peri.'],MPC_table['M'],MPC_table['H'],MPC_table['class'],MPC_table['Discovery']])
outTable

a,e,Incl.,Node,Peri.,M,H,class,Discovery
float64,float64,float64,float64,float64,float64,float64,int32,float64
45.334,0.132,27.2,174.4,331.1,245.892234777,7.4,1,56956.0
42.938,0.157,17.4,236.6,268.9,248.387738523,4.9,1,56952.0
40.436,0.074,17.8,98.3,267.1,7.25477415436,8.1,1,56932.0
41.148,0.106,19.3,113.2,301.2,319.134814572,8.0,1,56887.0
46.757,0.062,38.0,186.0,257.2,298.548790613,5.6,1,56888.0
45.999,0.28,20.7,144.7,242.7,-14.6992310179,6.7,1,56887.0
39.504,0.278,18.3,96.1,313.0,331.655797377,8.5,1,56890.0
48.359,0.252,26.3,75.8,293.5,-5.56169953011,6.5,1,56887.0
46.132,0.135,4.7,83.8,166.5,43.4807120863,6.0,1,56894.0


In [197]:
#Perpendicular Centaur Objects
outTable = Table([PCentaurTable['a'],PCentaurTable['e'],PCentaurTable['Incl.'],PCentaurTable['Node'],PCentaurTable['Peri.'],PCentaurTable['M'],PCentaurTable['H'],PCentaurTable['Precess'],PCentaurTable['Discovery']])
outTable

a,e,Incl.,Node,Peri.,M,H,Precess,Discovery
float64,float64,float64,float64,float64,float64,float64,float64,float64
1213.0,0.993,98.6,180.2,166.1,-0.136610808046,10.8,0.0,56312.0
1404.0,0.99,78.0,341.4,195.4,-0.109704792363,7.1,0.0,55979.0
484.0,0.987,143.9,176.0,179.1,-0.442011147845,10.2,0.0,55226.0
348.0,0.968,68.1,220.2,128.7,-0.689011534057,9.9,0.0,55007.0
323.0,0.971,140.8,136.2,132.9,-0.694197067899,10.6,0.0,55378.0


In [195]:
outTable.filled(-9999).write("AlignedKBOs.raw",format='ascii.no_header') #filled just in case, unnecessary I think

In [198]:
outTable.filled(-9999).write("PerpendicularCentaurs.raw",format='ascii.no_header') #filled just in case, unnecessary I think

In [187]:
#Aligned Objects
outTable = Table([FarTable['a'],FarTable['e'],FarTable['Incl.'],FarTable['Node'],FarTable['Peri.'],FarTable['M'],FarTable['H'],FarTable['Precess'],FarTable['Discovery']])
outTable

a,e,Incl.,Node,Peri.,M,H,Precess,Discovery
float64,float64,float64,float64,float64,float64,float64,float64,float64
325.0,0.888,29.6,67.6,316.5,-0.743726071964,8.6,0.86,56547.0
262.0,0.694,24.0,90.8,292.9,1.93910592116,4.0,0.27,56236.0
371.0,0.869,21.5,130.6,347.8,2.39236450606,6.5,0.06,55298.0
492.0,0.928,18.6,113.0,285.8,-0.228845249241,6.2,0.0,54376.0
320.0,0.852,25.6,66.0,327.2,-0.608210680871,6.4,-0.25,53315.0
507.0,0.85,11.9,144.5,311.5,357.594449758,1.6,0.13,52957.0


In [139]:
#Concatenate the two tables
from astropy.table import vstack
TNO_table = ascii.read('TNOs_MPC_Jan2016.raw')
Centaur_table = ascii.read('Centaurs_MPC_Jan2016.raw')
KBO_table=vstack([TNO_table,Centaur_table])

In [140]:
KBO_table

col1,col2,col3,col4,col5,col6,col7,col8,col9
float64,float64,float64,float64,float64,float64,float64,int32,float64
45.334,0.132,27.2,174.4,331.1,245.892234777,7.4,1,56956.0
42.938,0.157,17.4,236.6,268.9,248.387738523,4.9,1,56952.0
40.436,0.074,17.8,98.3,267.1,7.25477415436,8.1,1,56932.0
41.148,0.106,19.3,113.2,301.2,319.134814572,8.0,1,56887.0
46.757,0.062,38.0,186.0,257.2,298.548790613,5.6,1,56888.0
45.999,0.28,20.7,144.7,242.7,-14.6992310179,6.7,1,56887.0
39.504,0.278,18.3,96.1,313.0,331.655797377,8.5,1,56890.0
48.359,0.252,26.3,75.8,293.5,-5.56169953011,6.5,1,56887.0
46.132,0.135,4.7,83.8,166.5,43.4807120863,6.0,1,56894.0


In [141]:
KBO_table.filled(-9999).write("KBOs_MPC_Jan2016.raw",format='ascii.no_header') #filled just in case, unnecessary I think

In [101]:
Centaur_table

col1,col2,col3,col4,col5,col6,col7,col8,col9
float64,float64,float64,float64,float64,float64,float64,int32,float64
163.0,0.796,23.4,33.6,355.0,356.926712751,6.4,1,57285.0
14.36,0.539,9.5,12.8,30.7,234.241714641,11.5,1,57244.0
11.576,0.294,33.3,58.7,139.9,-135.634026796,12.3,1,57135.0
8.646,0.232,33.3,278.6,63.0,131.585595609,11.8,1,57135.0
19.219,0.325,37.7,6.3,237.7,238.801654566,8.3,1,57080.0
14.144,0.509,16.8,314.9,163.1,-101.197039995,12.6,1,57063.0
38.259,0.754,86.0,171.8,159.1,-23.3879805429,11.1,1,56933.0
25.82,0.176,15.5,117.2,359.2,256.411221286,9.5,1,56924.0
15.039,0.313,3.0,21.7,7.2,-98.3572966272,11.3,1,56919.0
